<td>   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a></td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/project_configuration/project_setup.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/project_configuration/project_setup.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Project Setup
* This notebok describes how to create and configure a project
* This is the same as creating a new project in the editor and going through all of the steps.

* When a user creates a project with client.create_project() the project is not ready for labeling.
    * An ontology must be set
    * A Batch must be created

In [ ]:
%pip install "labelbox[data]"

In [ ]:
import labelbox as lb
from labelbox.schema.quality_mode import QualityMode

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY = ""
client = lb.Client(api_key=API_KEY)

### Create Dataset
* Create dataset and attach data
* More details on attaching data can be found [here](https://github.com/Labelbox/labelbox-python/blob/master/examples/basics/data_rows.ipynb)

In [ ]:
dataset = client.create_dataset(name="project_setup_demo")
global_keys = ["id1", "id2", "id3", "id4"]
## Example image
uploads = []
# Generate data rows
for i in range(1, 5):
    uploads.append({
        "row_data":
            f"https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_000{i}.jpeg",
        "global_key":
            global_keys[i - 1],
    })
task = dataset.create_data_rows(uploads)
task.wait_till_done()
print("ERRORS: ", task.errors)
print("RESULT URL: ", task.result_url)

### Create Or Select an Ontology
* Optionally create an ontology or select from an existing one.
* More details on ontology management can be found [here](https://github.com/Labelbox/labelbox-python/blob/master/examples/basics/ontologies.ipynb)
   

In [ ]:
# Create
ontology_builder = lb.OntologyBuilder(
    tools=[lb.Tool(name="cat", tool=lb.Tool.Type.BBOX)])
ontology = client.create_ontology("project-setup-demo-ontology",
                                  ontology_builder.asdict())
# Select existing ontology
# ontology = client.get_ontology("<ontology_id>")
# ontology = existing_project.ontology()

### Create Project and Setup the Editor
* Setting up a project will add an ontology and will enable labeling to begin
* Creating batches will add all data_rows belonging to the dataset to the queue.

In [ ]:
batch_project = client.create_project(
    name="Project Setup Demo",
    quality_mode=QualityMode.
    Consensus,  # For benchmarks use quality_mode = QualityMode.Benchmark
    media_type=lb.MediaType.Image,
)

batch_project.setup_editor(ontology)

# Add data to your projects 

In [ ]:
## When creating a batch you can also setup the data rows priority
batch = batch_project.create_batch(
    "batch-demo-4",  # Each batch in a project must have a unique name
    global_keys=global_keys,  # A list of data rows or data row ids
    priority=5,  # priority between 1(Highest) - 5(lowest)
    consensus_settings={
        "number_of_labels": 2,
        "coverage_percentage": 1
    },
)
print("Batch: ", batch)

### Review

In [ ]:
# Note setup_complete will be None if it fails.
print(batch_project.setup_complete)
print(batch_project.ontology())
print([ds.name for ds in batch_project.batches()])

In [ ]:
print(f"https://app.labelbox.com/projects/{batch_project.uid}")

# Cleanup

In [ ]:
# batch_project.delete()
# dataset_project.delete()
# dataset.delete()